Implemented hit selection, events scatter, ion heatmap, electrons time of flight and ions time of flight

Additions:
+ merging multiple runs
+ spatial selection view
+ calibration choice of lines with corresponding zoomed-in views
+ calibration curve fit and apply with mq column

# Imports and functions

In [ ]:
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
#import ipywidgets as widgets
#from IPython.display import display